In [6]:
def makestar(stt,savename,date):
    from keras.models import load_model
    model = load_model('model/textlstm_0019-0.7360.hdf5')
    from konlpy.tag import Twitter, Kkma
    import pandas as pd
    import numpy as np
    import shutil
    import warnings
    warnings.filterwarnings(action='ignore') 
    stt = pd.DataFrame(stt)
    data = stt.iloc[:,0].values
    data = " ".join(data)
    kkma = Kkma()
    twitter = Twitter()
    l1 = twitter.nouns(data)
    l2 = kkma.nouns(data)
    l3 = list(set(l1).intersection(l2))
    import pickle
    with open('dict.pickle', 'rb') as handle:
        dic = pickle.load(handle)
    dic2 = {}
    for i in l3:
        if (i in dic.keys()):
            dic2[i] = dic[i]
    
    inputData = list(dic2.values())
    inputData2 = np.hstack([np.zeros(500-len(inputData)),inputData])
    inputData3 = np.vstack([inputData2,np.zeros(500)])
    
    proba = model.predict_proba(inputData3[0:1])[0]
    proba2 = ((proba-0.5)/2+0.5)**0.75
    
    import matplotlib.pyplot as plt
    from math import pi
    from matplotlib import font_manager, rc
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
     
    # Set data
    df = pd.DataFrame({
    'group': ['A','B','C','D','E'],
    'IT': [proba2[1], 1.5, 30, 4,1],
    '생산제조': [proba2[0], 10, 9, 34,1],
    '경영사무': [proba2[2], 0, 23, 24,1],
    '무역유통': [proba2[3], 31, 33, 14,1],
    '영업고객상담': [proba2[4], 15, 32, 14,1]
    })
     
    # number of variable
    categories=list(df)[1:]
    N = len(categories)
     
    # We are going to plot the first line of the data frame.
    # But we need to repeat the first value to close the circular graph:
    values=df.loc[0].drop('group').values.flatten().tolist()
    values += values[:1]
    values
     
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
     
    # Initialise the spider plot
    my_dpi = 500
    plt.figure(figsize=(1000/my_dpi, 1000/my_dpi), dpi=200)
    plt.title('직무 적합도 분석')
    ax = plt.subplot(111, polar=True)
     
    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='grey', size=7)
     
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([0.25,0.5,0.75,1], ["0.25","0.5","0.75"," 1"], color="grey", size=5)
    plt.ylim(0,1)
     
    ax.plot(angles, values, linewidth=1, linestyle='solid')
     
    # Fill area
    ax.fill(angles, values, 'b', alpha=0.2)
    arr = np.array([['IT',proba[1]],['생산제조',proba[0]],['경영사무',proba[2]],['무역유통',proba[3]],['영업고객상담',proba[4]]])
    subject = arr[:,0][(arr[:,1]).astype(float)==max(arr[:,1].astype(float))][0]
    import shutil
    shutil.copyfile('result/'+subject+'빈도.png','result/'+savename+date+'/'+subject+'빈도.png')
    shutil.copyfile('result/'+subject+'워드클라우드.jpg','result/'+savename+date+'/'+subject+'워드클라우드.jpg')
    plt.savefig('result/'+savename+date+'/'+savename+date+'star.jpg')
    print('stargraph saved in '+'result/'+savename+date+'/'+'star.jpg')
    print(subject+'워드클라우드.jpg is moved in result/'+savename+date)
    print(subject+'빈도.png is moved in result/'+savename+date)